In [109]:
import pandas as pd
import pymysql
from matplotlib import pyplot as plt
import numpy as np
import yaml
from os import path
import re
from sqlalchemy import create_engine

with open('../config.yml') as f:
    config = yaml.load(f)

project_dir = config['PROJECT']
project_db_dir = config['DATABASE']
con = pymysql.connect(host=config['MYSQL_HOST'], user=config['MYSQL_USER'], passwd=config['MYSQL_PASSWD'], 
                      db=config['MYSQL_DB'])
cur = con.cursor()
engine = create_engine("mysql+pymysql://{user}:{pw}@{server}/{db}"
                       .format(server=config['MYSQL_HOST'], user=config['MYSQL_USER'], pw=config['MYSQL_PASSWD'], 
                               db=config['MYSQL_DB']))

In [151]:
!bash -c "../scripts/dbsnp.sh"


Can't locate Bio/EnsEMBL/Registry.pm in @INC (you may need to install the Bio::EnsEMBL::Registry module) (@INC contains: /home/kaplanlab/vep/modules /home/kaplanlab/vep /opt/CilioGenics/domain/PfamScan /home/kaplanlab/perl5/lib/perl5/5.26.1/x86_64-linux-gnu-thread-multi /home/kaplanlab/perl5/lib/perl5/5.26.1 /home/kaplanlab/perl5/lib/perl5/x86_64-linux-gnu-thread-multi /home/kaplanlab/perl5/lib/perl5 /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.26.1 /usr/local/share/perl/5.26.1 /usr/lib/x86_64-linux-gnu/perl5/5.26 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.26 /usr/share/perl/5.26 /home/kaplanlab/perl5/lib/perl5/5.26.0 /home/kaplanlab/perl5/lib/perl5/5.26.0/x86_64-linux-gnu-thread-multi /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base) at /home/kaplanlab/vep/modules/Bio/EnsEMBL/VEP/BaseVEP.pm line 58.
BEGIN failed--compilation aborted at /home/kaplanlab/vep/modules/Bio/EnsEMBL/VEP/BaseVEP.pm line 58.
Compilation failed in require at /usr/share/perl/5.26/base.pm l

In [104]:
df = pd.read_csv(path.join(project_dir, 'dbSNP', 'result_only_hgvsp.tsv'), sep='\t')
df.loc[:, 'HGVSp'] = df['Extra'].apply(lambda x: re.sub(r'^(.*?)HGVSp=(.*?)(;|$)(.*?)', r'\2', x))
df.loc[:, 'Impact'] = df['Extra'].apply(lambda x: '' if 'IMPACT' not in x else
                                        re.sub(r'^(.*?)IMPACT=([^;]+)(.*?)$', r'\2', x))
df.drop(['Amino_acids', 'Codons', 'Existing_variation', 'Extra'], axis=1, inplace=True)

In [111]:
df.to_sql('dbsnp', engine, if_exists='replace', index=True)